In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('../data/house-prices.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
df.shape

(21613, 21)

### Pré-processamento

In [4]:
X = df.iloc[:, 5].values
X.shape

(21613,)

In [5]:
# transformando em matriz
X = X.reshape(-1, 1) # -1 não mexe nas linhas, add uma coluna
X.shape

(21613, 1)

In [6]:
y = df.iloc[:, 2:3].values # elimina o uso do reshape
y.shape

(21613, 1)

#### Escalonamento

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
X[:5]

/home/dstrb/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/dstrb/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.97983502],
       [ 0.53363434],
       [-1.42625404],
       [-0.13055006],
       [-0.43542158]])

In [9]:
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)
y[:5]

array([[-0.86671733],
       [-0.00568792],
       [-0.98084935],
       [ 0.17409044],
       [-0.08195753]])

In [10]:
colunas = [tf.feature_column.numeric_column('x', shape=[1])]
colunas

[_NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [11]:
regressor = tf.estimator.LinearRegressor(feature_columns=colunas)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9qou9id5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f38b96a7518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### Split base

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=.3)
X_treinamento.shape, X_teste.shape, y_treinamento.shape, y_teste.shape

((15129, 1), (6484, 1), (15129, 1), (6484, 1))

### Treinamento

In [14]:
funcao_treinamento = tf.estimator.inputs.numpy_input_fn(
    {'x': X_treinamento}, 
    y_treinamento,
    batch_size=32,
    num_epochs=None,
    shuffle=True
)

In [15]:
funcao_teste = tf.estimator.inputs.numpy_input_fn(
    {'x': X_teste}, 
    y_teste,
    batch_size=32,
    num_epochs=1000,
    shuffle=False
)

In [16]:
regressor.train(input_fn=funcao_treinamento, steps=10000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp9qou9id5/model.ckpt.
INFO:tensorflow:loss = 14.810031, step = 1
INFO:tensorflow:global_step/sec: 1695.21
INFO:tensorflow:loss = 13.583679, step = 101 (0.060 sec)
INFO:tensorflow:global_step/sec: 2116.71
INFO:tensorflow:loss = 16.130905, step = 201 (0.047 sec)
INFO:tensorflow:global_step/sec: 2027.34
INFO:tensorflow:loss = 15.972269, step = 301 (0.049 sec)
INFO:tensorflow:global_step/sec: 2129.05
INFO:tensorflow:loss = 10.641067, step = 401 (0.047 sec)
INFO

INFO:tensorflow:loss = 14.868628, step = 7001 (0.047 sec)
INFO:tensorflow:global_step/sec: 2076.26
INFO:tensorflow:loss = 5.186596, step = 7101 (0.048 sec)
INFO:tensorflow:global_step/sec: 2103.56
INFO:tensorflow:loss = 7.5183005, step = 7201 (0.048 sec)
INFO:tensorflow:global_step/sec: 2114.22
INFO:tensorflow:loss = 16.31725, step = 7301 (0.047 sec)
INFO:tensorflow:global_step/sec: 2046.58
INFO:tensorflow:loss = 6.507325, step = 7401 (0.049 sec)
INFO:tensorflow:global_step/sec: 2109.05
INFO:tensorflow:loss = 12.442099, step = 7501 (0.047 sec)
INFO:tensorflow:global_step/sec: 2112.33
INFO:tensorflow:loss = 10.512228, step = 7601 (0.048 sec)
INFO:tensorflow:global_step/sec: 2023.91
INFO:tensorflow:loss = 32.11124, step = 7701 (0.049 sec)
INFO:tensorflow:global_step/sec: 2154.46
INFO:tensorflow:loss = 11.549549, step = 7801 (0.047 sec)
INFO:tensorflow:global_step/sec: 2055.62
INFO:tensorflow:loss = 11.227213, step = 7901 (0.049 sec)
INFO:tensorflow:global_step/sec: 2116.52
INFO:tensorflo

### Métricas

In [17]:
metricas_treinamento = regressor.evaluate(input_fn=funcao_treinamento, steps = 10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-13-01:01:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9qou9id5/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Finished evaluation at 2019-02-13-01:01:29
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.5150046, global_step = 10000, label/mean = 0.0026129237, loss = 16.480146, prediction/mean = 0.0031624762
INFO:tensorflow:Saving 'checkpoint_pat

In [18]:
metricas_teste = regressor.evaluate(input_fn=funcao_teste, steps = 10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-13-01:02:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9qou9id5/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Finished evaluation at 2019-02-13-01:02:30
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.48799464, global_step = 10000, label/mean = -0.006607719, loss = 15.6158285, prediction/mean = 0.0042639184
INFO:tensorflow:Saving 'checkpoint_p

In [19]:
metricas_treinamento

{'average_loss': 0.5150046,
 'label/mean': 0.0026129237,
 'loss': 16.480146,
 'prediction/mean': 0.0031624762,
 'global_step': 10000}

In [20]:
metricas_teste

{'average_loss': 0.48799464,
 'label/mean': -0.006607719,
 'loss': 15.6158285,
 'prediction/mean': 0.0042639184,
 'global_step': 10000}

### Predict

In [21]:
import numpy as np

In [22]:
novas_casas = np.array([[800], [900], [1000]])

In [23]:
novas_casas = scaler_x.transform(novas_casas)

/home/dstrb/anaconda3/envs/tensor/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [24]:
funcao_previsao = tf.estimator.inputs.numpy_input_fn(
    {'x': novas_casas},
    shuffle=True
)

In [30]:
previsoes = regressor.predict(input_fn=funcao_previsao)
list(previsoes)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9qou9id5/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([-0.9647126], dtype=float32)},
 {'predictions': array([-0.8133886], dtype=float32)},
 {'predictions': array([-0.8890506], dtype=float32)}]

In [33]:
for p in regressor.predict(input_fn=funcao_previsao):
    print(p['predictions'], scaler_y.inverse_transform(p['predictions']))
    print()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9qou9id5/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[-0.8133886] [241477.98]

[-0.8890506] [213701.05]

[-0.9647126] [185924.11]

